In [59]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [60]:
%cd /content/drive/MyDrive/Hypertension_Final_Project/cleaned_file/2017

/content/drive/MyDrive/Hypertension_Final_Project/cleaned_file/2017


In [61]:
import pandas as pd
import numpy as np

In [62]:
pd.options.display.float_format = '{:20,.2f}'.format

In [63]:
year = 2017

In [64]:
BASE_PATH = "/content/drive/MyDrive/Hypertension_Final_Project/data_" + str(year) + '/'

In [65]:
BASE_PATH

'/content/drive/MyDrive/Hypertension_Final_Project/data_2017/'

# Hypertension Identification


## Blood Pressumre

In [66]:
# read sas file
blood_pressure = pd.read_sas(filepath_or_buffer = BASE_PATH + 'BPX.XPT')
# obtain Systolic_BP and Diastolic_BP data
blood_pressure_columns = ['SEQN', 'BPXPLS', 'BPXSY1', 'BPXDI1', 'BPXSY2', 'BPXDI2', 'BPXSY3', 'BPXDI3', 'BPXSY4', 'BPXDI4']
blood_pressure = blood_pressure[blood_pressure_columns]
# calculate average Systolic_BP and Diastolic_BP (average all 4 tests)
blood_pressure['Systolic_BP'] = blood_pressure.loc[:,['BPXSY1', 'BPXSY2', 'BPXSY3', 'BPXSY4']].mean(axis = 1)
blood_pressure['Diastolic_BP'] = blood_pressure.loc[:,['BPXDI1', 'BPXDI2', 'BPXDI3', 'BPXDI4']].mean(axis = 1)
# obtain clean the BP data
blood_pressure_final = blood_pressure.loc[:,['SEQN', 'Systolic_BP', 'Diastolic_BP']]

## Hypertension (Questionnaire)

In [67]:
# read questionnaire data
Hypertension = pd.read_sas(filepath_or_buffer = BASE_PATH + 'BPQ.XPT')
# preprocess the data
Hypertension_columns = ['SEQN', 'BPQ020', 'BPQ050A']
Hypertension = Hypertension[Hypertension_columns]

Told_High_blood_pressure
- 0：not been told that s/he has hypertension, also called high blood pressure
- 1：been told that s/he has hypertension, also called high blood pressure

In [68]:
Hypertension.loc[Hypertension['BPQ020'] == 2, 'Told_High_blood_pressure_dummy'] = 0
Hypertension.loc[Hypertension['BPQ020'] == 1, 'Told_High_blood_pressure_dummy'] = 1

In [69]:
# Taking Medicine for HBP
Hypertension.loc[Hypertension['BPQ050A'] == 2, 'Taking_Medicine_for_HBP_dummy'] = 0
Hypertension.loc[Hypertension['BPQ050A'] == 1, 'Taking_Medicine_for_HBP_dummy'] = 1

In [70]:
Hypertension.drop(columns = ['BPQ020', 'BPQ050A'], inplace = True)
Hypertension_combine = Hypertension.merge(blood_pressure_final, on = 'SEQN', how = 'outer')

In [71]:
Hypertension_final = Hypertension_combine[~((Hypertension_combine['Taking_Medicine_for_HBP_dummy'].isnull()) & (Hypertension_combine['Systolic_BP'].isnull()))]

In [72]:
Hypertension_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6904 entries, 0 to 9036
Data columns (total 5 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   SEQN                            6904 non-null   float64
 1   Told_High_blood_pressure_dummy  5707 non-null   float64
 2   Taking_Medicine_for_HBP_dummy   1942 non-null   float64
 3   Systolic_BP                     6717 non-null   float64
 4   Diastolic_BP                    6717 non-null   float64
dtypes: float64(5)
memory usage: 323.6 KB


Hypertension
- Systolic_BP > 130
- Diastolic_BP > 80
- Taking Medicine for HBP

In [73]:
Hypertension_final.loc[(Hypertension_final['Systolic_BP'] > 130) | (Hypertension_final['Diastolic_BP'] > 80) | (Hypertension_final['Taking_Medicine_for_HBP_dummy']) == 1, 'Hypertension_dummy'] = 1
Hypertension_final.loc[(Hypertension_final['Systolic_BP'] <= 130) | (Hypertension_final['Diastolic_BP'] <= 80), 'Hypertension_dummy'] = 0

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [74]:
Hypertension_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6904 entries, 0 to 9036
Data columns (total 6 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   SEQN                            6904 non-null   float64
 1   Told_High_blood_pressure_dummy  5707 non-null   float64
 2   Taking_Medicine_for_HBP_dummy   1942 non-null   float64
 3   Systolic_BP                     6717 non-null   float64
 4   Diastolic_BP                    6717 non-null   float64
 5   Hypertension_dummy              6869 non-null   float64
dtypes: float64(6)
memory usage: 377.6 KB


In [75]:
Hypertension_final = Hypertension_final[~Hypertension_final['Hypertension_dummy'].isnull()]

In [76]:
Hypertension_final['Hypertension_dummy'].value_counts()

0.00    5901
1.00     968
Name: Hypertension_dummy, dtype: int64

In [77]:
Hypertension_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6869 entries, 0 to 9036
Data columns (total 6 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   SEQN                            6869 non-null   float64
 1   Told_High_blood_pressure_dummy  5672 non-null   float64
 2   Taking_Medicine_for_HBP_dummy   1907 non-null   float64
 3   Systolic_BP                     6717 non-null   float64
 4   Diastolic_BP                    6717 non-null   float64
 5   Hypertension_dummy              6869 non-null   float64
dtypes: float64(6)
memory usage: 375.6 KB


In [78]:
Hypertension_final[Hypertension_final['Told_High_blood_pressure_dummy'] == 0]['Hypertension_dummy'].value_counts()

0.00    3226
1.00     356
Name: Hypertension_dummy, dtype: int64

In [79]:
Hypertension_final[Hypertension_final['Told_High_blood_pressure_dummy'] == 1]['Hypertension_dummy'].value_counts()

0.00    1481
1.00     609
Name: Hypertension_dummy, dtype: int64

In [80]:
Hypertension_final.to_csv('Hypertension_final.csv')

In [81]:
Hypertension_SEQN = Hypertension_final['SEQN']

In [82]:
Hypertension_SEQN.to_csv('Hyper_SEQN.csv')

# demographic

In [83]:
demographic = pd.read_sas(filepath_or_buffer = BASE_PATH + 'DEMO.XPT')

# respondents that were both interviewed and examined
demographic  = demographic[demographic['RIDSTATR'] == 2]

demographic.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8704 entries, 0 to 9253
Data columns (total 46 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   SEQN      8704 non-null   float64
 1   SDDSRVYR  8704 non-null   float64
 2   RIDSTATR  8704 non-null   float64
 3   RIAGENDR  8704 non-null   float64
 4   RIDAGEYR  8704 non-null   float64
 5   RIDAGEMN  555 non-null    float64
 6   RIDRETH1  8704 non-null   float64
 7   RIDRETH3  8704 non-null   float64
 8   RIDEXMON  8704 non-null   float64
 9   RIDEXAGM  3433 non-null   float64
 10  DMQMILIZ  5677 non-null   float64
 11  DMQADFC   527 non-null    float64
 12  DMDBORN4  8704 non-null   float64
 13  DMDCITZN  8701 non-null   float64
 14  DMDYRSUS  1848 non-null   float64
 15  DMDEDUC3  2167 non-null   float64
 16  DMDEDUC2  5265 non-null   float64
 17  DMDMARTL  5265 non-null   float64
 18  RIDEXPRG  1057 non-null   float64
 19  SIALANG   8704 non-null   float64
 20  SIAPROXY  8704 non-null   floa

In [84]:
demo_columns = ['SEQN', 'SDDSRVYR', 'RIAGENDR', 'RIDAGEYR', 'RIDRETH1', 'DMDBORN4', 'DMDMARTL', 'DMDEDUC3', 'DMDEDUC2', 'DMDHREDZ', 'INDFMIN2', 'INDFMPIR']
demographic = demographic[demo_columns]

## Marital_Status
- 1: Married/Living with partner
- 2: Widowed/Divorced/Separated
- 3: Never Married

In [85]:
demographic.loc[((demographic['DMDMARTL'] == 1) | (demographic['DMDMARTL'] == 6)), 'Marital_Status_dummy'] = 1
demographic.loc[((demographic['DMDMARTL'] == 2) | (demographic['DMDMARTL'] == 3) | (demographic['DMDMARTL'] == 4)), 'Marital_Status_dummy'] = 2
demographic.loc[demographic['DMDMARTL'] == 5 , 'Marital_Status_dummy'] = 3

In [86]:
demographic['Marital_Status_dummy'].value_counts()

1.00    3094
2.00    1218
3.00     948
Name: Marital_Status_dummy, dtype: int64

In [87]:
demographic.drop(columns = ['DMDMARTL'], inplace = True)

## Demographic: Education
- 1: Less than 9th grade
- 2: 9-11th grade(includes 12th grade with no diploma)
- 3: High school graduate/GED or equipment
- 4: some college or AA degreee
- 5: college graduate or above

In [88]:
demographic['Education_dummy'] = demographic['DMDEDUC2']
demographic.loc[(demographic['DMDEDUC3'] < 9) | (demographic['DMDEDUC3'] == 66), 'Education_dummy'] = 1
demographic.loc[(demographic['DMDEDUC3'] > 9) & (demographic['DMDEDUC3'] < 13), 'Education_dummy'] = 2
demographic.loc[(demographic['DMDEDUC3'] == 14) | (demographic['DMDEDUC3'] == 13), 'Education_dummy'] = 3
demographic.loc[demographic['DMDEDUC3'] == 15, 'Education_dummy'] = 4

In [89]:
demographic['Education_dummy'].replace({7: np.nan, 9:np.nan}, inplace = True)

In [90]:
demographic['Education_dummy'].value_counts()

1.00    1964
4.00    1756
3.00    1396
5.00    1261
2.00     899
Name: Education_dummy, dtype: int64

In [91]:
demographic = demographic.drop(columns = ['DMDEDUC3', 'DMDEDUC2'])

## Income & Income_to_Proverty_Ratio
Income_to_Proverty_Ratio
- 1: less than 5 (proverty)
- 2: >= 5


In [92]:
demographic['Income_dummy'] = demographic['INDFMIN2']
demographic.loc[demographic['INDFMIN2'] == 12, 'Income_dummy'] = 6
demographic.loc[demographic['INDFMIN2'] == 13, 'Income_dummy'] = 4
demographic.loc[demographic['INDFMIN2'] == 14, 'Income_dummy'] = 11
demographic.loc[demographic['INDFMIN2'] == 15, 'Income_dummy'] = 12
demographic['Income_dummy'].replace({77: np.nan, 99:np.nan}, inplace = True)

In [93]:
demographic['Income_dummy'].value_counts()

12.00    1486
 6.00    1179
 7.00     864
11.00     776
 4.00     642
 8.00     569
 5.00     526
 9.00     519
 3.00     432
10.00     391
 1.00     319
 2.00     273
Name: Income_dummy, dtype: int64

In [94]:
demographic.loc[demographic['INDFMPIR'] <= 1, 'Poverty_dummy'] = 1
demographic.loc[demographic['INDFMPIR'] > 1, 'Poverty_dummy'] = 0

In [95]:
demographic.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8704 entries, 0 to 9253
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   SEQN                  8704 non-null   float64
 1   SDDSRVYR              8704 non-null   float64
 2   RIAGENDR              8704 non-null   float64
 3   RIDAGEYR              8704 non-null   float64
 4   RIDRETH1              8704 non-null   float64
 5   DMDBORN4              8704 non-null   float64
 6   DMDHREDZ              8292 non-null   float64
 7   INDFMIN2              8338 non-null   float64
 8   INDFMPIR              7634 non-null   float64
 9   Marital_Status_dummy  5260 non-null   float64
 10  Education_dummy       7276 non-null   float64
 11  Income_dummy          7976 non-null   float64
 12  Poverty_dummy         7634 non-null   float64
dtypes: float64(13)
memory usage: 952.0 KB


In [96]:
demographic['DMDBORN4'].replace({7: np.nan, 9:np.nan}, inplace = True)

In [97]:
# demographic.loc[demographic['DMDBORN2'] == 1, 'Country_of_birth_dummy'] = 1
# demographic.loc[(demographic['DMDBORN2'] > 1) & (demographic['DMDBORN2'] < 7), 'Country_of_birth_dummy'] = 2


In [98]:
demographic.rename(columns={'RIAGENDR': 'gender_dummy', 'RIDAGEYR': 'age', 'RIDRETH1': 'race_dummy', 'DMDBORN4': 'Country_of_birth_dummy', 'DMDHREDU': 'Household_reference_education_dummy'}, inplace = True)

In [99]:
demographic.drop(columns = ['INDFMIN2', 'INDFMPIR'], inplace = True)

In [100]:
demographic.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8704 entries, 0 to 9253
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   SEQN                    8704 non-null   float64
 1   SDDSRVYR                8704 non-null   float64
 2   gender_dummy            8704 non-null   float64
 3   age                     8704 non-null   float64
 4   race_dummy              8704 non-null   float64
 5   Country_of_birth_dummy  8704 non-null   float64
 6   DMDHREDZ                8292 non-null   float64
 7   Marital_Status_dummy    5260 non-null   float64
 8   Education_dummy         7276 non-null   float64
 9   Income_dummy            7976 non-null   float64
 10  Poverty_dummy           7634 non-null   float64
dtypes: float64(11)
memory usage: 816.0 KB


In [101]:
len(demographic.columns) == 11

True

In [102]:
demographic.to_csv('cleaned_demographic.csv')

# Body Measure

In [103]:
# read body measure file from Examination Data
body_measure = pd.read_sas(filepath_or_buffer = BASE_PATH + 'BMX.XPT')

# select columns
body_columns = ['SEQN', 'BMXWT', 'BMXHT', 'BMXBMI', 'BMXARMC', 'BMXWAIST']
body_measure = body_measure[body_columns]

# rename the columns
body_measure.rename(columns={'BMXWT': 'Weight(kg)', 'BMXHT': 'Height(cm)', 'BMXBMI': 'BMI', 'BMXARMC': 'Arm_Circumference', 'BMXWAIST': 'Waist_Circumference'}, inplace = True)

In [104]:
body_measure.head()

,SEQN,Weight(kg),Height(cm),BMI,Arm_Circumference,Waist_Circumference
0,"93,703.00",13.70,88.60,17.50,16.20,48.20
1,"93,704.00",13.90,94.20,15.70,15.20,50.00
2,"93,705.00",79.50,158.30,31.70,32.00,101.80
3,"93,706.00",66.30,175.70,21.50,27.00,79.30
4,"93,707.00",45.40,158.40,18.10,21.50,64.10


In [105]:
len(body_measure.columns) == 6

True

In [106]:
body_measure.to_csv('cleaned_body_measure.csv')

# Other factors

## Consumer Behavior
The ratio is more important than the absolute value as the total spending amounts are different, therefore, the ratio is calcuated

In [107]:
# read sas file
consumer = pd.read_sas(filepath_or_buffer = BASE_PATH + 'CBQ.XPT')

consumer_col = ['SEQN', 'CBD071', 'CBD091', 'CBD111', 'CBD121', 'CBD131']

consumer = consumer[consumer_col]

# get the ratio of each spending category
ratio_cal = consumer.iloc[:,1:].div(consumer.iloc[:,1:].sum(axis=1),axis=0)

ratio_cal['Spending_on_food'] = ratio_cal['CBD071'] + ratio_cal['CBD111']
ratio_cal.rename(columns = {'CBD091': 'Spending_on_nonfood', 'CBD121': 'Spending_on_eating_out', 'CBD131': 'Spending_on_carryout_deliverred'}, inplace = True)

consumer_final = pd.concat([consumer, ratio_cal], axis=1)
consumer_final = consumer_final.drop(columns = ['CBD071', 'CBD091', 'CBD111', 'CBD121', 'CBD131'])

In [108]:
consumer_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9254 entries, 0 to 9253
Data columns (total 5 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   SEQN                             9254 non-null   float64
 1   Spending_on_nonfood              8773 non-null   float64
 2   Spending_on_eating_out           8780 non-null   float64
 3   Spending_on_carryout_deliverred  8777 non-null   float64
 4   Spending_on_food                 8780 non-null   float64
dtypes: float64(5)
memory usage: 361.6 KB


In [109]:
print(len(consumer_final.columns) == 5) 

True


In [110]:
consumer_final.to_csv('cleaned_consumer_behavior.csv')

## Alcohol drink

In [111]:
alcohol = pd.read_sas(filepath_or_buffer = BASE_PATH + 'ALQ.XPT')

In [112]:
alcohol.loc[alcohol['ALQ121'] == 0, 'alcohol_drink_frequency_dummy'] = 0
alcohol.loc[(alcohol['ALQ121'] < 10) | (alcohol['ALQ121'] >= 7), 'alcohol_drink_frequency_dummy'] = 1
alcohol.loc[(alcohol['ALQ121'] > 0) | (alcohol['ALQ121'] < 7), 'alcohol_drink_frequency_dummy'] = 2

In [113]:
alcohol['alcohol_drink_per_day'] = alcohol['ALQ130']
alcohol['alcohol_drink_per_day'].replace({777: np.nan, 999:np.nan}, inplace = True)

In [114]:
alcohol_columns = ['SEQN', 'alcohol_drink_frequency_dummy', 'alcohol_drink_per_day']

In [115]:
alcohol_final = alcohol[alcohol_columns]

In [116]:
len(alcohol_final.columns) == 3

True

In [117]:
alcohol_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5533 entries, 0 to 5532
Data columns (total 3 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   SEQN                           5533 non-null   float64
 1   alcohol_drink_frequency_dummy  4545 non-null   float64
 2   alcohol_drink_per_day          3489 non-null   float64
dtypes: float64(3)
memory usage: 129.8 KB


In [118]:
alcohol_final.to_csv('cleaned_alcohol.csv')

## Diet behavior

In [119]:
Diet_behavior = pd.read_sas(filepath_or_buffer = BASE_PATH + 'DBQ.XPT')

Diet_behavior_cols = ['SEQN', 'DBQ197', 'DBQ223A',  'DBQ223B', 'DBQ223C', 'DBQ223D', 'DBQ223E', 'DBQ223U', 'DBD895', 'DBD905', 'DBD910']
Diet_behavior = Diet_behavior[Diet_behavior_cols]

# get the milk type by assigning row-wise minimum value of all milk type columns
Diet_behavior['milk_type_dummy'] = Diet_behavior[['DBQ223A',  'DBQ223B', 'DBQ223C', 'DBQ223D', 'DBQ223E', 'DBQ223U']].min(axis=1)
Diet_behavior.loc[Diet_behavior['milk_type_dummy'] == 99, 'milk_type_dummy'] = np.nan


Diet_behavior.drop(columns = ['DBQ223A',  'DBQ223B', 'DBQ223C', 'DBQ223D', 'DBQ223E', 'DBQ223U'], inplace = True)

Diet_behavior['milk_product_consumption_freq_dummy'] = Diet_behavior['DBQ197']
Diet_behavior.loc[Diet_behavior['DBQ197'] == 4, 'milk_product_consumption_freq_dummy'] = 2

# number of meals not home prepared
Diet_behavior['eat_outside'] = Diet_behavior['DBD895']
Diet_behavior.loc[Diet_behavior['DBD895'] == 5555, 'eat_outside'] = 22
Diet_behavior.loc[Diet_behavior['DBD895'] > 5555, 'eat_outside'] = np.nan

# number of ready-to-eat foods in past 30 days
Diet_behavior['ready_to_eat_food'] = Diet_behavior['DBD905']
Diet_behavior.loc[Diet_behavior['DBD905'] == 6666, 'ready_to_eat_food'] = 100
Diet_behavior.loc[Diet_behavior['DBD905'] > 6666, 'ready_to_eat_food'] = np.nan

# of frozen meals/pizza in past 30 days
Diet_behavior['frozen_food'] = Diet_behavior['DBD910']
Diet_behavior.loc[Diet_behavior['DBD910'] == 6666, 'frozen_food'] = 100
Diet_behavior.loc[Diet_behavior['DBD910'] > 6666, 'frozen_food'] = np.nan

Diet_behavior.drop(columns = ['DBQ197', 'DBD895', 'DBD905', 'DBD910'], inplace = True)

In [120]:
len(Diet_behavior.columns) == 6

True

In [121]:
Diet_behavior.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9254 entries, 0 to 9253
Data columns (total 6 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   SEQN                                 9254 non-null   float64
 1   milk_type_dummy                      7514 non-null   float64
 2   milk_product_consumption_freq_dummy  8897 non-null   float64
 3   eat_outside                          8820 non-null   float64
 4   ready_to_eat_food                    8807 non-null   float64
 5   frozen_food                          8816 non-null   float64
dtypes: float64(6)
memory usage: 433.9 KB


In [122]:
Diet_behavior.to_csv('cleaned_diet_behavior.csv')

## Mental Health

In [123]:
mental_health = pd.read_sas(filepath_or_buffer = BASE_PATH + 'DPQ.XPT')

mental_health_cols = ['SEQN', 'DPQ030', 'DPQ040', 'DPQ050', 'DPQ060']
mental_health = mental_health[mental_health_cols]

mental_health_cols = ['DPQ030', 'DPQ040', 'DPQ050', 'DPQ060']

for col in mental_health_cols:
  mental_health.loc[(mental_health[col] < 1) , col + '_1'] = 'low'
  mental_health.loc[(mental_health[col] == 1) | (mental_health[col] == 2), col + '_1'] = 'mild'
  mental_health.loc[(mental_health[col] == 3) , col + '_1'] = 'severe'

mental_health = mental_health.rename(columns = {'DPQ030_1': 'trouble_sleeping_dummy', 'DPQ040_1': 'feeling_tired_dummy', 'DPQ050_1':'poor_appetite_dummy', 'DPQ060_1':'feeling_bad_dummy'})
mental_health.drop(columns=['DPQ030', 'DPQ040', 'DPQ050', 'DPQ060'], inplace = True)


In [124]:
len(mental_health.columns) == 5

True

In [125]:
mental_health.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5533 entries, 0 to 5532
Data columns (total 5 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   SEQN                    5533 non-null   float64
 1   trouble_sleeping_dummy  5086 non-null   object 
 2   feeling_tired_dummy     5085 non-null   object 
 3   poor_appetite_dummy     5087 non-null   object 
 4   feeling_bad_dummy       5083 non-null   object 
dtypes: float64(1), object(4)
memory usage: 216.3+ KB


In [126]:
mental_health.to_csv('cleaned_mental_health.csv')

## diabetes

In [127]:
diabetes = pd.read_sas(filepath_or_buffer = BASE_PATH + 'DIQ.XPT')
diabetes.loc[(diabetes['DIQ010']==9),'DIQ010']=np.nan
diabetes.loc[(diabetes['DIQ050']==9),'DIQ010']=np.nan
diabetes.loc[(diabetes['DIQ160']==9),'DIQ010']=np.nan
diabetes=diabetes[['SEQN','DIQ010','DIQ050','DIQ160']]

dia_cols=['SEQN','have_diabetes_1_dummy','Taking_insulin_now_1_dummy','have_prediabetes_1_dummy']
diabetes.columns=dia_cols
clean_diabetes=diabetes

In [128]:
clean_diabetes.to_csv('cleaned_diabetes.csv')

## Health Insurance

In [129]:
insurance = pd.read_sas(filepath_or_buffer = BASE_PATH + 'HIQ.XPT')

insurance.loc[(insurance['HIQ011']==9),'HIQ011']=np.nan
insurance.loc[(insurance['HIQ011']==7),'HIQ011']=np.nan
insurance.loc[(insurance['HIQ270']==9),'HIQ270']=np.nan
insurance.loc[(insurance['HIQ270']==7),'HIQ270']=np.nan

insurance=insurance[['SEQN','HIQ011','HIQ270']]

insurance_cols=['SEQN','Covered_by_health_insurance_1_dummy','plans_cover_prescriptions_1_dummy']
insurance.columns=insurance_cols
clean_insurance=insurance

In [130]:
clean_insurance.to_csv('cleaned_insurance.csv')

## Current_health_Status

In [131]:
health_status = pd.read_sas(filepath_or_buffer = BASE_PATH + 'HSQ.XPT')

health_status.loc[(health_status['HSQ500']==9),'HSQ500']=np.nan
health_status.loc[(health_status['HSQ500']==7),'HSQ500']=np.nan
health_status.loc[(health_status['HSQ510']==9),'HSQ510']=np.nan
health_status.loc[(health_status['HSQ510']==7),'HSQ510']=np.nan
health_status.loc[(health_status['HSQ520']==9),'HSQ520']=np.nan
health_status=health_status[['SEQN','HSQ500','HSQ510','HSQ520']]

health_status_cols=['SEQN','head_cold_or_chest_cold_dummy','stomach_or_intestinal_illness_dummy','flu_pneumonia_ear_infection_dummy']
health_status.columns=health_status_cols

In [132]:
health_status.to_csv('cleaned_health_status.csv')

## Occupation

In [133]:
occupation = pd.read_sas(filepath_or_buffer = BASE_PATH + 'OCQ.XPT')

occupation=occupation[['SEQN','OCD150']]
occupation.loc[(occupation['OCD150']!=1),'OCD150']=0
occupation['OCD150'].value_counts()
occ_cols=['SEQN','have_job_1_dummy']
occupation.columns=occ_cols

In [134]:
occupation.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6161 entries, 0 to 6160
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   SEQN              6161 non-null   float64
 1   have_job_1_dummy  6161 non-null   float64
dtypes: float64(2)
memory usage: 96.4 KB


In [135]:
occupation.to_csv('cleaned_occupation.csv')

## Medical Condition

In [136]:
med_condition = pd.read_sas(filepath_or_buffer = BASE_PATH + 'MCQ.XPT')

med_condition=med_condition[['SEQN','MCQ010','MCQ053','MCQ092']]

med_condition.loc[(med_condition['MCQ010']==9),'MCQ010']=np.nan
med_condition.loc[(med_condition['MCQ053']==7),'MCQ053']=np.nan
med_condition.loc[(med_condition['MCQ053']==9),'MCQ053']=np.nan
med_condition.loc[(med_condition['MCQ092']==7),'MCQ092']=np.nan
med_condition.loc[(med_condition['MCQ092']==9),'MCQ092']=np.nan

med_cols=['SEQN','have_asthma_1_dummy','treatment_for_anemia_1_dummy','blood_transfusion_1_dummy']
med_condition.columns=med_cols

In [137]:
med_condition.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8897 entries, 0 to 8896
Data columns (total 4 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   SEQN                          8897 non-null   float64
 1   have_asthma_1_dummy           8888 non-null   float64
 2   treatment_for_anemia_1_dummy  8882 non-null   float64
 3   blood_transfusion_1_dummy     7796 non-null   float64
dtypes: float64(4)
memory usage: 278.2 KB


In [138]:
med_condition.to_csv('cleaned_med_condition.csv')

## Physical Activity

In [139]:
physical_act = pd.read_sas(filepath_or_buffer = BASE_PATH + 'PAQ.XPT')

physical_act=physical_act[['SEQN','PAQ605','PAQ620','PAQ635','PAQ650','PAQ665']]
physical_act.loc[(physical_act['PAQ605']==9),'PAQ605']=np.nan
physical_act.loc[(physical_act['PAQ620']==9),'PAQ620']=np.nan

physical_act_cols=['SEQN','Vigorous_work_activity_1_dummy','Moderate work activity_1_dummy','Walk_or_bicycle_1_dummy','Vigorous_recreational_activities_1_dummy','Moderate_recreational_activities_1_dummy']
physical_act.columns=physical_act_cols

In [140]:
physical_act.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5856 entries, 0 to 5855
Data columns (total 6 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   SEQN                                      5856 non-null   float64
 1   Vigorous_work_activity_1_dummy            5850 non-null   float64
 2   Moderate work activity_1_dummy            5851 non-null   float64
 3   Walk_or_bicycle_1_dummy                   5856 non-null   float64
 4   Vigorous_recreational_activities_1_dummy  5856 non-null   float64
 5   Moderate_recreational_activities_1_dummy  5856 non-null   float64
dtypes: float64(6)
memory usage: 274.6 KB


In [141]:
physical_act.to_csv('cleaned_physical_act.csv')

## Sleeping disorder

In [142]:
sleep_disorder = pd.read_sas(filepath_or_buffer = BASE_PATH + 'SLQ.XPT')


sleep_disorder.loc[(sleep_disorder['SLQ050']==9),'SLQ050']=np.nan

sleep_disorder=sleep_disorder[['SEQN', 'SLD012', 'SLQ050']]
sleep_cols=['SEQN', 'Sleep_hours', 'had_trouble_sleeping_1_dummy']
sleep_disorder.columns=sleep_cols

In [143]:
sleep_disorder.head()

,SEQN,Sleep_hours,had_trouble_sleeping_1_dummy
0,"93,705.00",8.00,2.00
1,"93,706.00",10.50,2.00
2,"93,708.00",8.00,2.00
3,"93,709.00",7.00,2.00
4,"93,711.00",7.00,1.00


In [144]:
# # sleep hour
# sleep_disorder['hour'] = pd.to_numeric(sleep_disorder['Usual_sleep_time'].str[:2])
# sleep_disorder['min'] = pd.to_numeric(sleep_disorder['Usual_sleep_time'].str[3:])
# sleep_disorder = sleep_disorder[~(sleep_disorder['hour'] > 30)]
# sleep_disorder['min_decimal'] = sleep_disorder['min'] / 60
# sleep_disorder['final_sleep_time'] = sleep_disorder['hour'] + sleep_disorder['min_decimal']


# # Wake hour
# sleep_disorder['wake_hour'] = pd.to_numeric(sleep_disorder['Usual_wake_time'].str[:2])
# sleep_disorder['wake_min'] = pd.to_numeric(sleep_disorder['Usual_wake_time'].str[3:])
# sleep_disorder = sleep_disorder[~(sleep_disorder['wake_hour'] > 30)]
# sleep_disorder['wake_min_decimal'] = sleep_disorder['wake_min'] / 60
# sleep_disorder['final_wake_time'] = sleep_disorder['wake_hour'] + sleep_disorder['wake_min_decimal']

In [145]:
sleep_disorder.columns

Index(['SEQN', 'Sleep_hours', 'had_trouble_sleeping_1_dummy'], dtype='object')

In [146]:
# sleep_disorder.drop(columns = ['Usual_sleep_time', 'Usual_wake_time', 'hour', 'min', 'min_decimal',\
#                                'wake_hour', 'wake_min', 'wake_min_decimal'], inplace = True)

In [147]:
# sleep_disorder.head()

In [148]:
sleep_disorder.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6161 entries, 0 to 6160
Data columns (total 3 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   SEQN                          6161 non-null   float64
 1   Sleep_hours                   6113 non-null   float64
 2   had_trouble_sleeping_1_dummy  6156 non-null   float64
dtypes: float64(3)
memory usage: 144.5 KB


In [149]:
sleep_disorder.to_csv('cleaned_sleep_disorder.csv')

## Smoking

In [150]:
smoking = pd.read_sas(filepath_or_buffer = BASE_PATH + 'SMQ.XPT')

smoking=smoking[['SEQN','SMQ020']]
smoking_cols=['SEQN','>100_cigarettes_inlife_1_dummy']
smoking.columns=smoking_cols

In [151]:
smoking.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6724 entries, 0 to 6723
Data columns (total 2 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   SEQN                            6724 non-null   float64
 1   >100_cigarettes_inlife_1_dummy  5856 non-null   float64
dtypes: float64(2)
memory usage: 105.2 KB


In [152]:
smoking.to_csv('cleaned_smoking.csv')

## Smoking - Household Smokers

In [153]:
# HH_smokers = pd.read_sas(filepath_or_buffer = BASE_PATH + 'SMQFAM.XPT')

# HH_smokers.loc[(HH_smokers['SMD460']==7.770000e+02),'SMD460']=np.nan
# HH_smokers.loc[(HH_smokers['SMD460']==9.990000e+02 ),'SMD460']=np.nan

# HH_smokers=HH_smokers[['SEQN','SMD460']]
# HH_smokers_cols=['SEQN','HH_smoke_number_dummy']
# HH_smokers.columns=HH_smokers_cols

In [154]:
# HH_smokers.info()

In [155]:
# HH_smokers.to_csv('cleaned_HH_smoker.csv')

# Diary View

In [159]:
d1 = pd.read_sas(filepath_or_buffer = BASE_PATH + 'DR1TOT.XPT')
d1.loc[(d1['WTDRD1']<0.01),'WTDRD1']=np.nan
d1.loc[(d1['WTDR2D']<0.01),'WTDR2D']=np.nan
d1.loc[(d1['DR1DRSTZ']==2),'DR1DRSTZ']=np.nan
d1.loc[(d1['DR1DRSTZ']==5),'DR1DRSTZ']=np.nan
d1.loc[(d1['DR1LANG']==3),'DR1LANG']=2
d1.loc[(d1['DR1LANG']==5),'DR1LANG']=4
d1.loc[(d1['DR1LANG']==6),'DR1LANG']=4
d1.loc[(d1['DBQ095Z']==99),'DBQ095Z']=np.nan
d1.loc[(d1['DBD100']==9),'DBD100']=np.nan
d1.loc[(d1['DRQSPREP']==9),'DRQSPREP']=np.nan
# d1.loc[(d1['DR1TWS']==99),'DR1TWS']=np.nan
d1.loc[(d1['DRD340']==9),'DRD340']=np.nan
d1.loc[(d1['DRD340']==7),'DRD340']=np.nan

d1=d1.rename({'DRABF':'Breast-fed_infantdummy','DRDINT':'Number_of_days_of_intake_dummy','DR1LANG':'language_dummy','DBQ095Z':'salt_type_dummy','DBD100':'salt_freq_dummy','DRQSPREP':'salt_prep_dummy','DRQSDIET':'special_diet_dummy','DR1TWS':'tap_water_source_dummy','DRD340':'Shellfish_eaten_1_dummy','DRD350A':'clams_eaten_1_dummy','DRD350B':'crabs_eanten_1_dummy','DRD350C':'crayfish_eaten-1_dummy','DRD350D':'lobsters+eaten_1_dummy','DRD350E':'mussels_eaten_1_dummy','DRD350F':'oysters_eaten_1_dummy'})
d1=d1.drop(columns={'DR1EXMER','DR1DAY', 'DRQSDT1','DRQSDT2','DRQSDT3','DRQSDT4','DRQSDT5','DRQSDT6','DRQSDT7','DRQSDT8','DRQSDT9','DRQSDT10','DRQSDT91','DR1_300','DRD370JQ','DRD370KQ','DRD370LQ','DRD370NQ','DRD370OQ','DRD370PQ','DRD370QQ','DRD370RQ','DRD370SQ','DRD370UQ','DRD370V'})

/usr/local/lib/python3.7/dist-packages/pandas/io/sas/sas_xport.py:475: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df[x] = v


In [160]:
d1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8704 entries, 0 to 8703
Columns: 143 entries, SEQN to DRD370U
dtypes: float64(143)
memory usage: 9.6 MB


In [161]:
d1.to_csv('cleaned_dietary_data.csv')

In [163]:
d2 = pd.read_sas(filepath_or_buffer = BASE_PATH + 'DR2TOT.XPT')
d2.loc[(d1['WTDRD1']<0.01),'WTDRD1']=np.nan
d2.loc[(d1['WTDR2D']<0.01),'WTDR2D']=np.nan
d2.loc[(d1['DR1DRSTZ']==2),'DR1DRSTZ']=np.nan
d2.loc[(d1['DR1DRSTZ']==5),'DR1DRSTZ']=np.nan
d2.loc[(d1['DR1LANG']==3),'DR1LANG']=2
d2.loc[(d1['DR1LANG']==5),'DR1LANG']=4
d2.loc[(d1['DR1LANG']==6),'DR1LANG']=4
d2.loc[(d1['DBQ095Z']==99),'DBQ095Z']=np.nan
d2.loc[(d1['DBD100']==9),'DBD100']=np.nan
d2.loc[(d1['DRQSPREP']==9),'DRQSPREP']=np.nan
# d2.loc[(d1['DR1TWS']==99),'DR1TWS']=np.nan
d2.loc[(d1['DRD340']==9),'DRD340']=np.nan
d2.loc[(d1['DRD340']==7),'DRD340']=np.nan
d2=d2.rename({'DRABF':'Breast-fed_infantdummy','DRDINT':'Number_of_days_of_intake_dummy','DR1LANG':'language_dummy','DBQ095Z':'salt_type_dummy','DBD100':'salt_freq_dummy','DRQSPREP':'salt_prep_dummy','DRQSDIET':'special_diet_dummy','DR1TWS':'tap_water_source_dummy','DRD340':'Shellfish_eaten_1_dummy','DRD350A':'clams_eaten_1_dummy','DRD350B':'crabs_eanten_1_dummy','DRD350C':'crayfish_eaten-1_dummy','DRD350D':'lobsters+eaten_1_dummy','DRD350E':'mussels_eaten_1_dummy','DRD350F':'oysters_eaten_1_dummy'})
d2.to_csv('cleaned_dietary_data_2.csv')